
# 🎬 Vision Classroom Demos (All-in-One): DINOv2 + SAM‑2

此 Notebook 集成两个课堂演示：

- **DINOv2**：自监督视觉表征 → 聚类可视化 + 最近邻检索
- **SAM‑2**（脚手架/可选）：交互式分割（需模型权重，演示区提供占位与说明）

> 建议方式：把此 `.ipynb` 上传到 GitHub，然后在 `README` 放一个 **Open in Colab** 按钮，课堂上一键打开运行。


## 0. 环境安装与检查

In [ ]:

!pip -q install --upgrade timm torchvision scikit-learn pillow matplotlib

import torch, timm, os, io, requests
import numpy as np
import matplotlib.pyplot as plt

from timm.data import resolve_data_config, create_transform
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

print("Torch:", torch.__version__)
print("timm :", timm.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)



---
## Part A — DINOv2：特征可视化 & 最近邻检索（可直接运行 ✅）

目标：
1. 提取多张图片的 DINOv2 表征，并用 **PCA** 做二维可视化（同类更接近）。
2. 选一张图做 **最近邻检索**（用余弦相似度找最像的几张）。


In [ ]:

# 1) 加载 DINOv2（timm 提供的预训练权重，推荐 vit_small 变体以加快推理）
model = timm.create_model("vit_small_patch14_dinov2", pretrained=True)
model.eval().to(device)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)
print("Transform config:", config)


In [ ]:

# 2) 准备演示图片：可以替换为你自己的 URL 或本地路径
IMAGE_URLS = [
    "/content/cat.jpg",    # 猫
    "/content/dog.jpg",    # 狗
    "/content/car.jpg",    # 汽车
    "/content/plane.jpg",  # 飞机
]

def load_image(path_or_url):
    if path_or_url.startswith("http"):
        r = requests.get(path_or_url, timeout=15)
        r.raise_for_status()
        img = Image.open(io.BytesIO(r.content)).convert("RGB")
    else:
        img = Image.open(path_or_url).convert("RGB")
    return img

images = [load_image(u) for u in IMAGE_URLS]
print(f"Loaded {len(images)} images.")


In [ ]:

# 3) 提取特征：使用 forward_head(..., pre_logits=True) 获取分类头前的表征
import torch.nn.functional as F

@torch.no_grad()
def extract_feature(img: Image.Image):
    x = transform(img).unsqueeze(0).to(device)         # [1, C, H, W]
    feats = model.forward_features(x)                  # dict/tensor by model
    emb = model.forward_head(feats, pre_logits=True)   # [1, D]
    emb = F.normalize(emb, dim=-1)                     # 归一化便于余弦相似度
    return emb.squeeze(0).cpu().numpy()                # [D]

features = np.stack([extract_feature(img) for img in images], axis=0)  # [N, D]
print("Feature shape:", features.shape)


In [ ]:

# 4) PCA 2D 可视化
pca = PCA(n_components=2, random_state=0)
xy = pca.fit_transform(features)  # [N, 2]

plt.figure(figsize=(6, 5))
plt.scatter(xy[:, 0], xy[:, 1], s=80)
for i, (x, y) in enumerate(xy):
    plt.text(x + 0.01, y + 0.01, f"img{i}", fontsize=10)
plt.title("DINOv2 Features (PCA 2D)")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.grid(True); plt.show()


In [ ]:

# 5) 最近邻检索：给定 query，展示 Top-k 相似图片
def show_img(img, title=None):
    plt.imshow(img); plt.axis("off")
    if title: plt.title(title)

def knn_retrieval(query_idx=0, topk=3):
    sims = cosine_similarity(features[query_idx:query_idx+1], features)[0]  # [N]
    order = np.argsort(-sims)  # 从大到小
    print(f"[Query] img{query_idx}  → Top-{topk} similar indices:", order[:topk+1])

    cols = topk + 1
    plt.figure(figsize=(3.2*cols, 3.5))
    plt.subplot(1, cols, 1); show_img(images[query_idx], title=f"Query (img{query_idx})")
    shown = 1
    for idx in order:
        if idx == query_idx: 
            continue
        plt.subplot(1, cols, shown+1)
        show_img(images[idx], title=f"Top{shown} (img{idx})\ncos={sims[idx]:.3f}")
        shown += 1
        if shown > topk: 
            break
    plt.tight_layout(); plt.show()

# 试一试：把 0 改成 1/2/3 等，看检索结果
knn_retrieval(query_idx=0, topk=3)



> **课堂讲解建议（~3分钟）**  
> 1. 说明 DINOv2 是自监督视觉表征；  
> 2. 展示 PCA 图：同类样本更靠近；  
> 3. 运行 KNN 检索，多换几张 query 看 Top‑k；  
> 4. 鼓励同学上传现场图片做即场检索。



---
## Part B — SAM‑2：交互式分割（脚手架 / 可选）

> 说明：SAM‑2 需要加载较大的模型权重（数百 MB），为了让本 Notebook 保持单文件、小体积、即开即用，下面提供 **占位与演示框架**：
>
> - 你只需将官方 **checkpoint 下载链接** 填入下方变量，或把权重放到 Colab/本地路径；
> - 运行安装与加载单元，即可用点/框提示完成分割；
> - 若课堂网络不稳定，建议提前在 Colab 里预运行并缓存（或录屏作为备份）。


In [ ]:

# 1) （可选）安装 SAM‑2
# 提示：如果你已有运行环境，可注释掉此步
# 尝试从 GitHub 安装（如失败，请改用你本地/内网镜像或提前装好环境）
try:
    import sam2  # noqa: F401  # 若已安装则跳过
    print("sam2 已存在，跳过安装。")
except Exception as e:
    print("准备安装 sam2（如失败，请根据你的环境调整安装方式）")
    !pip -q install 'git+https://github.com/facebookresearch/sam2.git' || true


In [ ]:

# 2) 设置权重路径/URL（请替换为官方 checkpoints 的直链，或上传到 /content/ 目录）
SAM2_CKPT_URL = "REPLACE_WITH_OFFICIAL_SAM2_CHECKPOINT_URL"  # TODO: 替换为真实链接
SAM2_CKPT_PATH = "/content/sam2_checkpoint.pt"               # 下载保存路径（可改）

# 如果你已把权重上传到 Colab 或本地，直接把 SAM2_CKPT_PATH 指向该文件即可，跳过下载。
DOWNLOAD_IF_URL_SET = SAM2_CKPT_URL.startswith("http")
if DOWNLOAD_IF_URL_SET:
    import requests, io
    from pathlib import Path
    try:
        print("Downloading SAM‑2 checkpoint ...")
        r = requests.get(SAM2_CKPT_URL, timeout=60)
        r.raise_for_status()
        Path(SAM2_CKPT_PATH).write_bytes(r.content)
        print("Checkpoint saved to:", SAM2_CKPT_PATH)
    except Exception as e:
        print("⚠️ 下载失败：", e)
        print("请手动上传权重到", SAM2_CKPT_PATH, "或提供可访问的下载链接。")
else:
    print("跳过下载，使用本地路径：", SAM2_CKPT_PATH)


In [ ]:

# 3) 载入模型（API 随版本可能有差异，以下为伪代码式示例框架，便于你替换为实际调用）
# 注意：请根据官方 README 示例替换为对应的构建/加载接口。

import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def overlay_mask(image_pil, mask_bool, alpha=0.6):
    img = np.array(image_pil).astype(np.float32)
    mask = mask_bool.astype(np.float32)[..., None]
    color = np.array([0, 255, 0], dtype=np.float32)  # 绿色遮罩
    vis = img * (1 - alpha * mask) + color * (alpha * mask)
    return vis.astype(np.uint8)

# 占位：加载任意一张图片
IMG_URL = "https://images.pexels.com/photos/1108099/pexels-photo-1108099.jpeg"
img = Image.open(requests.get(IMG_URL, stream=True).raw).convert("RGB")

print("占位提示：请参考官方 API，将 SAM‑2 predictor 初始化与推理代码粘贴到此处。")
print("示例伪代码：")
print("""
from sam2 import build_sam2_predictor  # 请以实际 API 为准
predictor = build_sam2_predictor(checkpoint=SAM2_CKPT_PATH, device=device)

# 以单点提示为例：point_coords: np.ndarray[[x, y]]
point_coords = np.array([[320, 240]]); point_labels = np.array([1])  # 正提示点
masks = predictor.predict(image=img, point_coords=point_coords, point_labels=point_labels)
""")

# 下面展示如何把 mask 可视化到原图（masks[0] 为 bool/0-1 掩膜）
# masks = predictor.predict(...)
# vis = overlay_mask(img, masks[0])
# plt.figure(figsize=(6,4)); plt.imshow(vis); plt.axis("off"); plt.show()

plt.figure(figsize=(6,4)); plt.imshow(img); plt.title("占位图像（请在上方集成 SAM‑2 推理）"); plt.axis("off"); plt.show()



> **将来你只需做两件事：**  
> 1. 把 `SAM2_CKPT_URL` 换成官方 checkpoint 的可下载直链（或把权重上传并设置 `SAM2_CKPT_PATH`）。  
> 2. 将你所用版本的 **官方 Predictor 初始化与 `predict` 示例** 粘贴到上面的占位代码处，并把 `masks` 可视化即可。  
>
> 这样，本 Notebook 仍保持单文件结构，便于 **“一个文件上 GitHub”** 与 **课堂一键演示**。
